# Dataobjects by example

## Basic

In [2]:
from recordclass import dataobject, clsconfig
import sys

### Empty dataobject

In [2]:
class Empty(dataobject):
     pass

In [3]:
Empty()

Empty()

In [4]:
print('Sizeof:', sys.getsizeof(Empty()))

Sizeof: 16


### Simple 2d data point

In [5]:
class Point(dataobject):
    x:int
    y:int

In [25]:
p1 = Point(1,2)
p2 = Point(x=1, y=2)
print(p1, p2)
p1 == p2

Point(x=1, y=2) Point(x=1, y=2)


True

But
> Only instances of the same class may be equal.

By default instances are mutable:

In [15]:
p = Point(1,2)
p.x = 100
p.y = 200
print(p)

Point(x=100, y=200)


There is an option `readonly=True` for immutability of the instances:

In [21]:
class PointRO(dataobject):
    x:int
    y:int
    __options__ = {'readonly':True}

In [23]:
p = PointRO(1,2)
try:
    p.x = 100
except TypeError as e:
    print(e)

item is readonly


For fast instance creation one may use `fast_new` option:

In [7]:
class FastPoint(dataobject, fast_new=True):
    x:int
    y:int

The following timings explain the advantage of `fast_new` option:

In [8]:
%timeit l1 = [Point(i,i) for i in range(100000)]
%timeit l2 = [FastPoint(i,i) for i in range(100000)]

25.6 ms ± 2.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
10.4 ms ± 426 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


By default dataobject subclasses are iterable:

In [9]:
[v for v in Point(1,2)]

[1, 2]

If one want to disable iterability of the Point instances, then one have to use `clsconfig` adapter and `iterable=False` argument:

In [10]:
@clsconfig(iterable=False)
class PointNI(dataobject):
    x:int
    y:int

In [11]:
try:
    [v for v in PointNI(1,2)]
except TypeError as e:
    print(e)

'PointNI' object is not iterable


### RGB colors

Define class for representation of RGB colors:

In [12]:
class RGB(dataobject):
    r:int = 0
    g:int = 0
    b:int = 0

One may define default values:

In [13]:
p1 = RGB(128, 64)
p2 = RGB(r=128, g=64)
print(p1)
print(p1 == p2)

RGB(r=128, g=64, b=0)
True


Onw have to note that fields without default values have to appear first. Here is invalid definition when the field without default value appears after the field with default value:

In [14]:
try:
    class PointInvalidDefaults(dataobject):
        x:int = 0
        y:int
except TypeError as e:
    print(e)

The field without default value appear after the field with default value


In [1]:
from recordclass import RecordClass

In [6]:
@clsconfig(gc=False)
class Point(RecordClass):
    x:int
    y:int
        

In [7]:
sys.getsizeof(Point(1,2))

32

In [15]:
class A(dataobject, fast_new=True):
    x:int
    y:int
    
    def __init__(self, x, y):
        if not (isinstance(x, int) and isinstance(y, int)):
            raise TypeError('x,y are not int')
            
class B:
    x:int
    y:int
    __slots__ = 'x', 'y'
    
    def __init__(self, x, y):
        if not (isinstance(x, int) and isinstance(y, int)):
            raise TypeError('x,y are not int')
        self.x = x
        self.y = y


In [16]:
%timeit l1 = [A(1,2) for i in range(1000000)]

184 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit l2 = [B(1,2) for i in range(1000000)]

231 ms ± 6.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
